In [1]:
# Definicion de la funcion IOU y de la funcion de perdida para YOLOv3
# Incluyo ambas cosas en la misma celda para que se puedan probar juntas

import torch
import torch.nn as nn
import sys
import os

# PASO 1: Definicion de la función intersection_over_union

def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calcula la Intersection Over Union (IoU) entre bounding boxes.

    Args:
        boxes_preds (tensor): Bounding boxes predichas de forma (N, 4) o (batch_size, 4)
                            donde N es el número de cajas o 1 si es una sola caja.
                            Formato de las cajas: (x, y, w, h) o (x1, y1, x2, y2).
        boxes_labels (tensor): Bounding boxes ground truth de forma (N, 4) o (batch_size, 4).
        box_format (str): Formato de las cajas de entrada.
                        "midpoint" si es (x_center, y_center, width, height)
                        "corners" si es (x1, y1, x2, y2).

    Returns:
        tensor: IoU para cada par de cajas, de forma (N, 1) o (batch_size, 1).
    """

    if box_format == "midpoint":
        # Convertir de (x_center, y_center, width, height) a (x1, y1, x2, y2)
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    elif box_format == "corners":
        # Asumir que ya están en (x1, y1, x2, y2)
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]
    else:
        raise ValueError("box_format debe ser 'midpoint' o 'corners'")

    # Calcular las coordenadas del rectángulo de intersección
    x1_inter = torch.max(box1_x1, box2_x1)
    y1_inter = torch.max(box1_y1, box2_y1)
    x2_inter = torch.min(box1_x2, box2_x2)
    y2_inter = torch.min(box1_y2, box2_y2)

    # Calcular el área de intersección
    intersection = (x2_inter - x1_inter).clamp(0) * \
                (y2_inter - y1_inter).clamp(0)

    # Calcular el área de cada bounding box
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    # Calcular el IoU
    union = box1_area + box2_area - intersection + 1e-6 # Añadir epsilon para evitar división por cero
    iou = intersection / union

    return iou

# PASO 2: Definición de la clase YOLOv3Loss

class YOLOv3Loss(nn.Module):
    def __init__(self, anchors, num_classes, img_size=(416, 416), 
                lambda_coord=1.0, lambda_noobj=1.0, lambda_obj=1.0, lambda_class=1.0, 
                ignore_iou_threshold=0.5): # Umbral para ignorar anchors en noobj loss
        super().__init__()
        self.anchors = anchors 
        self.num_classes = num_classes
        self.img_size = img_size
        self.lambda_coord = lambda_coord 
        self.lambda_noobj = lambda_noobj 
        self.lambda_obj = lambda_obj     
        self.lambda_class = lambda_class 
        self.ignore_iou_threshold = ignore_iou_threshold 

        self.mse = nn.MSELoss() 
        self.bce = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0])) 

    def forward(self, predictions, targets):
        obj_loss = 0
        noobj_loss = 0
        box_loss = 0
        class_loss = 0 

        for scale_idx, prediction in enumerate(predictions):
            prediction = prediction.permute(0, 2, 3, 1).reshape(
                prediction.shape[0], prediction.shape[2], prediction.shape[3], 3, self.num_classes + 5
            )
            
            pred_x_y = prediction[..., 0:2] 
            pred_w_h = prediction[..., 2:4]                 
            pred_obj = prediction[..., 4:5]               
            pred_class = prediction[..., 5:]               

            N, grid_h, grid_w, num_anchors, _ = prediction.shape
            
            anchors_current_scale = torch.tensor(self.anchors[scale_idx], device=targets.device).reshape(1, 1, 1, num_anchors, 2)
            
            target_obj_mask = torch.zeros((N, grid_h, grid_w, num_anchors), dtype=torch.float32, device=targets.device)
            target_noobj_mask = torch.ones((N, grid_h, grid_w, num_anchors), dtype=torch.float32, device=targets.device)
            
            tx = torch.zeros((N, grid_h, grid_w, num_anchors), device=targets.device)
            ty = torch.zeros((N, grid_h, grid_w, num_anchors), device=targets.device)
            tw = torch.zeros((N, grid_h, grid_w, num_anchors), device=targets.device) 
            th = torch.zeros((N, grid_h, grid_w, num_anchors), device=targets.device) 
            
            target_class_one_hot = torch.zeros((N, grid_h, grid_w, num_anchors, self.num_classes), dtype=torch.float32, device=targets.device) 

            for box_idx in range(targets.shape[0]):
                img_id, class_id, x_gt_norm, y_gt_norm, w_gt_norm, h_gt_norm = targets[box_idx].tolist()
                img_id = int(img_id) 

                x_center_grid = x_gt_norm * grid_w
                y_center_grid = y_gt_norm * grid_h
                
                cell_x = int(x_center_grid)
                cell_y = int(y_center_grid)

                if cell_x >= grid_w or cell_y >= grid_h or cell_x < 0 or cell_y < 0:
                    continue
                
                w_gt_abs_pixels = w_gt_norm * self.img_size[0]
                h_gt_abs_pixels = h_gt_norm * self.img_size[1]
                
                gt_box_dims = torch.tensor([0, 0, w_gt_abs_pixels, h_gt_abs_pixels], device=targets.device)

                anchor_boxes_for_iou = torch.zeros((num_anchors, 4), device=targets.device)
                anchor_boxes_for_iou[:, 2] = anchors_current_scale[0,0,0,:,0] 
                anchor_boxes_for_iou[:, 3] = anchors_current_scale[0,0,0,:,1] 
                
                ious = intersection_over_union(
                    gt_box_dims.unsqueeze(0), 
                    anchor_boxes_for_iou,     
                    box_format="corners"      
                ) 
                
                best_iou_anchor_idx = torch.argmax(ious).item() 
                
                target_obj_mask[img_id, cell_y, cell_x, best_iou_anchor_idx] = 1.0 
                target_noobj_mask[img_id, cell_y, cell_x, best_iou_anchor_idx] = 0.0 
                
                tx[img_id, cell_y, cell_x, best_iou_anchor_idx] = x_center_grid - cell_x
                ty[img_id, cell_y, cell_x, best_iou_anchor_idx] = y_center_grid - cell_y
                
                tw[img_id, cell_y, cell_x, best_iou_anchor_idx] = torch.log(w_gt_abs_pixels / anchors_current_scale[0,0,0,best_iou_anchor_idx,0] + 1e-16) 
                th[img_id, cell_y, cell_x, best_iou_anchor_idx] = torch.log(h_gt_abs_pixels / anchors_current_scale[0,0,0,best_iou_anchor_idx,1] + 1e-16) 
                
                target_class_one_hot[img_id, cell_y, cell_x, best_iou_anchor_idx, int(class_id)] = 1.0 

                for anchor_idx_other, iou_val in enumerate(ious[0]): 
                    if anchor_idx_other == best_iou_anchor_idx:
                        continue 
                    
                    if iou_val > self.ignore_iou_threshold:
                        target_noobj_mask[img_id, cell_y, cell_x, anchor_idx_other] = 0.0 
            
            loss_x = self.bce(pred_x_y[..., 0][target_obj_mask.bool()], tx[target_obj_mask.bool()])
            loss_y = self.bce(pred_x_y[..., 1][target_obj_mask.bool()], ty[target_obj_mask.bool()])

            loss_w = self.mse(pred_w_h[..., 0][target_obj_mask.bool()], tw[target_obj_mask.bool()]) 
            loss_h = self.mse(pred_w_h[..., 1][target_obj_mask.bool()], th[target_obj_mask.bool()]) 
            
            box_loss += (loss_x + loss_y + loss_w + loss_h)

            loss_obj = self.bce(pred_obj[target_obj_mask.bool()], target_obj_mask[target_obj_mask.bool()].float().unsqueeze(-1))
            loss_noobj = self.bce(pred_obj[target_noobj_mask.bool()], target_noobj_mask[target_noobj_mask.bool()].float().unsqueeze(-1))
            
            obj_loss += loss_obj
            noobj_loss += loss_noobj

            loss_class = self.bce(pred_class[target_obj_mask.bool()], target_class_one_hot[target_obj_mask.bool()])
            class_loss += loss_class

        total_loss = (
            self.lambda_coord * box_loss
            + self.lambda_obj * obj_loss
            + self.lambda_noobj * noobj_loss
            + self.lambda_class * class_loss
        )
        return total_loss, {"box_loss": box_loss, "obj_loss": obj_loss, "noobj_loss": noobj_loss, "class_loss": class_loss}




In [2]:
# Ejemplo de uso de la función intersection_over_union

# import torch
# Asegúrate de que la función intersection_over_union esté accesible,
# por ejemplo, si la tienes en un archivo llamado 'utils.py', la importarías así:
# from utils import intersection_over_union

# --- Ejemplo 1: Cajas superpuestas (formato 'corners') ---
print("--- Ejemplo 1: Cajas superpuestas (formato 'corners') ---")
box1 = torch.tensor([[0.0, 0.0, 10.0, 10.0]]) # Caja 1: (0,0) a (10,10)
box2 = torch.tensor([[5.0, 5.0, 15.0, 15.0]]) # Caja 2: (5,5) a (15,15)
# Intersección: (5,5) a (10,10) -> Ancho 5, Alto 5 -> Área = 25
# Área box1 = 100, Área box2 = 100
# Unión = 100 + 100 - 25 = 175
# IoU esperado = 25 / 175 = 0.1428...

iou_result = intersection_over_union(box1, box2, box_format="corners")
print(f"IoU (corners): {iou_result.item()}")
# Debería ser aproximadamente 0.142857

# --- Ejemplo 2: Cajas idénticas (formato 'midpoint') ---
print("\n--- Ejemplo 2: Cajas idénticas (formato 'midpoint') ---")
# Caja 1: centro (5,5), ancho 10, alto 10 -> (0,0) a (10,10)
box1_mid = torch.tensor([[5.0, 5.0, 10.0, 10.0]])
# Caja 2: centro (5,5), ancho 10, alto 10 -> (0,0) a (10,10)
box2_mid = torch.tensor([[5.0, 5.0, 10.0, 10.0]])
# IoU esperado = 1.0 (cajas idénticas)

iou_result_mid = intersection_over_union(box1_mid, box2_mid, box_format="midpoint")
print(f"IoU (midpoint, idénticas): {iou_result_mid.item()}")
# Debería ser 1.0

# --- Ejemplo 3: Cajas sin superposición (formato 'corners') ---
print("\n--- Ejemplo 3: Cajas sin superposición (formato 'corners') ---")
box1_no_overlap = torch.tensor([[0.0, 0.0, 10.0, 10.0]])
box2_no_overlap = torch.tensor([[11.0, 11.0, 20.0, 20.0]])
# IoU esperado = 0.0

iou_result_no_overlap = intersection_over_union(box1_no_overlap, box2_no_overlap, box_format="corners")
print(f"IoU (corners, sin superposición): {iou_result_no_overlap.item()}")
# Debería ser 0.0

# --- Ejemplo 4: Cajas con una dimensión cero (para probar clamp) ---
print("\n--- Ejemplo 4: Cajas con una dimensión cero (para probar clamp) ---")
box1_zero_width = torch.tensor([[0.0, 0.0, 0.0, 10.0]]) # Ancho cero
box2_zero_width = torch.tensor([[0.0, 0.0, 5.0, 10.0]])
# IoU esperado = 0.0 (o un valor muy pequeño debido al epsilon)

iou_result_zero_width = intersection_over_union(box1_zero_width, box2_zero_width, box_format="corners")
print(f"IoU (corners, ancho cero): {iou_result_zero_width.item()}")
# Debería ser 0.0 (o cercano a cero)

# --- Ejemplo 5: Cajas con diferentes tamaños (formato 'midpoint') ---
print("\n--- Ejemplo 5: Cajas con diferentes tamaños (formato 'midpoint') ---")
# box1: centro (10,10), w=10, h=10 -> (5,5) a (15,15)
box1_diff = torch.tensor([[10.0, 10.0, 10.0, 10.0]])
# box2: centro (10,10), w=5, h=5 -> (7.5,7.5) a (12.5,12.5)
box2_diff = torch.tensor([[10.0, 10.0, 5.0, 5.0]])
# box1_area = 100, box2_area = 25
# Intersección = 25 (box2 está completamente dentro de box1)
# Unión = 100 + 25 - 25 = 100
# IoU esperado = 25 / 100 = 0.25

iou_result_diff = intersection_over_union(box1_diff, box2_diff, box_format="midpoint")
print(f"IoU (midpoint, diferentes tamaños): {iou_result_diff.item()}")
# Debería ser 0.25

--- Ejemplo 1: Cajas superpuestas (formato 'corners') ---
IoU (corners): 0.1428571492433548

--- Ejemplo 2: Cajas idénticas (formato 'midpoint') ---
IoU (midpoint, idénticas): 1.0

--- Ejemplo 3: Cajas sin superposición (formato 'corners') ---
IoU (corners, sin superposición): 0.0

--- Ejemplo 4: Cajas con una dimensión cero (para probar clamp) ---
IoU (corners, ancho cero): 0.0

--- Ejemplo 5: Cajas con diferentes tamaños (formato 'midpoint') ---
IoU (midpoint, diferentes tamaños): 0.25


In [3]:
# Ejemplo de uso de la funcion loss de YOLOv3

if __name__ == "__main__":
    # --- Configuración para la prueba ---
    NUM_CLASSES = 3 # RBC, WBC, Platelets
    IMG_SIZE = (416, 416)
    BATCH_SIZE = 2 # Tamaño de lote para la prueba

    # Anchors para YOLOv3 (ejemplo de COCO para 416x416)
    # Debes usar los anchors que sean adecuados para tu dataset
    #ANCHORS = [
    #    [(10, 13), (16, 30), (33, 23)],  # Escala 0 (grid 13x13)
    #    [(30, 61), (62, 45), (59, 119)], # Escala 1 (grid 26x26)
    #    [(116, 90), (156, 198), (373, 326)], # Escala 2 (grid 52x52)
    #]
    
    ANCHORS = [
    [(227, 210), (179, 155), (124, 111)],  # Anchors para la escala más grande (stride 32, detecta objetos grandes)
    [(105, 113), (104, 96), (80, 109)],    # Anchors para la escala media (stride 16, detecta objetos medianos)
    [(112, 75), (87, 82), (39, 38)]        # Anchors para la escala más pequeña (stride 8, detecta objetos pequeños)
]
    
    # Convertir anchors a tensores de PyTorch para pasarlos a la pérdida
    ANCHORS_TENSOR = [torch.tensor(a) for a in ANCHORS]

    # --- Instanciar la función de pérdida ---
    loss_fn = YOLOv3Loss(
        anchors=ANCHORS, # Pasamos la lista de Python, la clase la convertirá a tensor
        num_classes=NUM_CLASSES,
        img_size=IMG_SIZE,
        lambda_coord=1.0,
        lambda_noobj=1.0,
        lambda_obj=1.0,
        lambda_class=1.0,
        ignore_iou_threshold=0.5
    )

    # --- Crear predicciones dummy (simulando la salida del modelo) ---
    # La salida del modelo son 3 tensores, uno por cada escala.
    # Formato: (N, 3 * (5 + num_classes), Grid_H, Grid_W)

    # Predicciones para la escala 0 (grid 13x13)
    pred0_grid_h, pred0_grid_w = IMG_SIZE[0] // 32, IMG_SIZE[1] // 32 # 13x13
    predictions0 = torch.randn(BATCH_SIZE, 3 * (5 + NUM_CLASSES), pred0_grid_h, pred0_grid_w)

    # Predicciones para la escala 1 (grid 26x26)
    pred1_grid_h, pred1_grid_w = IMG_SIZE[0] // 16, IMG_SIZE[1] // 16 # 26x26
    predictions1 = torch.randn(BATCH_SIZE, 3 * (5 + NUM_CLASSES), pred1_grid_h, pred1_grid_w)

    # Predicciones para la escala 2 (grid 52x52)
    pred2_grid_h, pred2_grid_w = IMG_SIZE[0] // 8, IMG_SIZE[1] // 8 # 52x52
    predictions2 = torch.randn(BATCH_SIZE, 3 * (5 + NUM_CLASSES), pred2_grid_h, pred2_grid_w)

    predictions_dummy = [predictions0, predictions1, predictions2]

    # --- Crear targets dummy (simulando las ground truth boxes) ---
    # Formato: (num_true_boxes_in_batch, 6) -> (image_idx, class_id, x_norm, y_norm, w_norm, h_norm)
    # x,y,w,h normalizados globalmente [0,1]
    targets_dummy = torch.tensor([
        [0, 0, 0.5, 0.5, 0.1, 0.1],   # Imagen 0, Clase 0, centro, caja pequeña
        [0, 1, 0.2, 0.8, 0.05, 0.05], # Imagen 0, Clase 1, abajo-izquierda, caja muy pequeña
        [0, 2, 0.7, 0.3, 0.2, 0.2],   # Imagen 0, Clase 2, arriba-derecha, caja mediana
        [1, 0, 0.1, 0.1, 0.3, 0.3],   # Imagen 1, Clase 0, arriba-izquierda, caja grande
        [1, 1, 0.9, 0.9, 0.08, 0.08], # Imagen 1, Clase 1, abajo-derecha, caja pequeña
    ], dtype=torch.float32)

    # Mover a la GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    predictions_dummy = [p.to(device) for p in predictions_dummy]
    targets_dummy = targets_dummy.to(device)
    loss_fn.to(device)

    # --- Ejecutar la función de pérdida ---
    print("\n--- Ejecutando la función de pérdida con datos dummy ---")
    total_loss, individual_losses = loss_fn(predictions_dummy, targets_dummy)

    print(f"\nPérdida Total: {total_loss.item():.4f}")
    print(f"  Pérdida de Cajas (box_loss): {individual_losses['box_loss'].item():.4f}")
    print(f"  Pérdida de Objeto (obj_loss): {individual_losses['obj_loss'].item():.4f}")
    print(f"  Pérdida de No-objeto (noobj_loss): {individual_losses['noobj_loss'].item():.4f}")
    print(f"  Pérdida de Clase (class_loss): {individual_losses['class_loss'].item():.4f}")

    # --- Verificaciones básicas (opcional) ---
    # Si las pérdidas son NaN o inf, algo anda mal.
    if torch.isnan(total_loss) or torch.isinf(total_loss):
        print("\n¡Advertencia! La pérdida total es NaN o Inf. Esto indica un problema numérico.")
    else:
        print("\nLa función de pérdida se ejecutó exitosamente con valores numéricos válidos.")

    print("\n--- Fin de la prueba ---")
    print("\n--- Fin de la prueba ---")



--- Ejecutando la función de pérdida con datos dummy ---

Pérdida Total: 22.9507
  Pérdida de Cajas (box_loss): 16.0573
  Pérdida de Objeto (obj_loss): 2.1578
  Pérdida de No-objeto (noobj_loss): 2.4027
  Pérdida de Clase (class_loss): 2.3329

La función de pérdida se ejecutó exitosamente con valores numéricos válidos.

--- Fin de la prueba ---

--- Fin de la prueba ---
